## Extraction of Notes from Downloaded Sites
1. Form lists of phrases for each text
2. Repair words with star (ex.: ж*па -> жопа)

In [1]:
from collections import defaultdict, Counter
import jsonlines
import os
import re
from tqdm import tqdm

data_dir = '../data'

In [2]:
def look_soup(dd, link):
    print('\n'.join([abstract[0] for abstract in dd[link] if abstract[1] == 'text']))
    
def look_listphrases(listphrases):
    print('\n***********\n'.join(['\n'.join(paragraph) for paragraph in listphrases]))

#### prohiphop.org

In [3]:
dd_prohiphop = defaultdict(list)

with jsonlines.open(os.path.join(data_dir,'site_soup_prohiphop.jsonl')) as fd:
    for obj in tqdm(fd):
        dd_prohiphop[obj['link']] += obj['soup']

131it [00:00, 4429.90it/s]


texts didn't download :(

#### webrap.info

In [4]:
dd_webrap = defaultdict(list)

with jsonlines.open(os.path.join(data_dir,'site_soup_webrap.jsonl')) as fd:
    for obj in tqdm(fd):
        dd_webrap[obj['link']] += obj['soup']

3152it [00:00, 5601.72it/s]


In [5]:
list_text_uchastnikov_versusa = sorted([link for link in dd_webrap if re.match('^\S*/text-uchastnikov-versusa\S+$', link) != None])
print(len(list_text_uchastnikov_versusa))

247


In [6]:
list_text_pesen = sorted([link for link in dd_webrap if re.match('^\S*/text-pesen\S+/\S+$', link) != None])
print(len(list_text_pesen))

1725


In [8]:
def extraction(soup):
    list_text = []
    list_tmp = []
    fl_israp = 0
    fl_newparagraph = 0
    for abstract in soup:
        fl_isphrase = 1
        text = abstract[0]
        block = abstract[2]
        if re.match('^Просмотров: \d+$', text) != None:
            fl_israp = 1
            fl_isphrase = 0
        elif (fl_israp == 1) and ('a:' in block) and (len(list_text) > 0):
            fl_israp = 0
            fl_isphrase = 0
        elif re.match('^\S*h\d:\S*$', block) != None:
            fl_isphrase = 0
            fl_newparagraph = 1
            
        if (fl_israp == 1) and (fl_isphrase == 1):
            if fl_newparagraph == 1:
                if len(list_tmp) > 0:
                    list_text.append(list_tmp)
                    list_tmp = [text]
                else:
                    list_tmp = [text]
                fl_newparagraph = 0
            else:
                list_tmp.append(text)
    if len(list_tmp) > 0:
        list_text.append(list_tmp)
    return list_text

In [9]:
dd_text_uchastnikov_versusa = {link:extraction(dd_webrap[link]) for link in tqdm(list_text_uchastnikov_versusa)}

100%|██████████| 247/247 [00:00<00:00, 1512.97it/s]


In [10]:
def extraction(soup):
    list_text = []
    list_tmp = []
    fl_israp = 0
    fl_newparagraph = 0
    num_paragraph = -999
    for abstract in soup:
        fl_isphrase = 1
        text = abstract[0]
        block = abstract[2]
        if re.match('^Просмотров: \d+$', text) != None:
            fl_israp = 1
            fl_isphrase = 0
        elif 'a:' in block:
            fl_israp = 0
            fl_isphrase = 0
        elif ('припев' in text.lower()) and (len(text) <= 20):
            fl_isphrase = 0
            
        if re.match('^\S*p:\S*$', block) != None:
            num_paragraph_curr = int(re.findall(r'p:\d+', block)[0].split(':')[1])
            if num_paragraph_curr > num_paragraph:
                num_paragraph = num_paragraph_curr
                fl_newparagraph = 1
        
        if (fl_israp == 1) and (fl_isphrase == 1):
            if fl_newparagraph == 1:
                if len(list_tmp) > 0:
                    list_text.append(list_tmp)
                    list_tmp = [text]
                else:
                    list_tmp = [text]
                fl_newparagraph = 0
            else:
                list_tmp.append(text)
    if len(list_tmp) > 0:
        list_text.append(list_tmp)
    
    if len(list_text) >= 10:
        list_text = [[phrase for paragraph in list_text for phrase in paragraph]]
    
    return list_text

In [11]:
dd_text_pesen = {link:extraction(dd_webrap[link]) for link in tqdm(list_text_pesen)}

100%|██████████| 1725/1725 [00:00<00:00, 2062.57it/s]


In [12]:
dd_webrap_texts = dd_text_uchastnikov_versusa.copy()

for link, list_text in dd_text_pesen.items():
    dd_webrap_texts[link] = list_text
print(len(dd_webrap_texts))

1972


In [16]:
with jsonlines.open(os.path.join(data_dir, 'texts_webrap.jsonl'), mode='w') as fd:
    for link, list_texts in dd_webrap_texts.items():
        if len(list_texts) > 0:
            fd.write({'link':link, 'list_texts':list_texts})

#### rap-text.ru